### Imports

In [2]:
import sys
import time
import random
import math as mt
from IPython.display   import clear_output
from termcolor         import colored
from enum              import Enum
from __future__        import annotations
from typing            import TypeVar, Optional, Generic, List, Callable, NamedTuple, Set, Dict
from IPython.display   import Markdown, display
T = TypeVar("T")
print(sys.version)

3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]


### Implementation of Data Structures

In [3]:
#------------------------------------------------------------------------------------------------------------------------------------------------
# Data Structures
#------------------------------------------------------------------------------------------------------------------------------------------------
class Stack(Generic[T]):
    def __init__(self) -> None:
        self.container: List[T] = []
    def __repr__(self) -> str:
        return repr(self.container)
    @property
    def isempty(self) -> bool:
        return not self.container
    def push(self, item: T) -> None:
        self.container.append(item)
    def pop(self) -> T:
        return self.container.pop()
#------------------------------------------------------------------------------------------------------------------------------------------------
class DQueue(Generic[T]):
    def __init__(self) -> None:
        self.container: List[T] = []
    def __repr__(self) -> str:
        return repr(self.container)
    @property
    def isempty(self) -> bool:
        return not self.container
    def push(self, item: T) -> None:
        self.container.append(item)
    def pop(self) -> T:
        return self.container.pop(0)
#------------------------------------------------------------------------------------------------------------------------------------------------
class PQueue(Generic[T]):
    def __init__(self) -> None:
        self.container: List[T] = []
    def __repr__(self) -> str:
        return repr(self.container)
    @property
    def isempty(self) -> bool:
        return not self.container
    def push(self, item: T) -> None:
        self.container.append(item)
        self.container.sort(key=lambda x: (x.cost+x.heur), reverse=True)
    
    def pop(self) -> T:
        return self.container.pop()
#------------------------------------------------------------------------------------------------------------------------------------------------
class Node(Generic[T]):
    def __init__(self,xy_loc: T, parent: Optional[Node], cost: float=1.0, heur: float=0.0) -> None:
        self.xy_loc: T              = xy_loc
        self.parent: Optional[Node] = parent
        self.cost:   float          = cost
        self.heur:   float          = heur
    def __lt__(self, versus: Node) -> bool:
        return (self.cost+self.heur) < (versus.cost + versus.heur)


### Implementation of Search Algorithms

In [4]:
#------------------------------------------------------------------------------------------------------------------------------------------------
# Algorithms
#------------------------------------------------------------------------------------------------------------------------------------------------
class Search_Utility():
    def __init__(self) -> None: 
        pass
#------------------------------------------------------------------------------------------------------------------------------------------------
    def dfs(self, strt: T, trgt: Callable[[T], bool], next: Callable[[T], List[T]]) -> Optional[Node[T]]:        
        front: Stack[Node[T]] = Stack()
        visit:        Set[T]  = {strt}
        front.push(Node(strt,None))
        while not front.isempty:                  #while we can explore
            curr: Node[T] = front.pop()           #get last in node
            if trgt(curr.xy_loc): return curr     #goal found
            for child in next(curr.xy_loc):       #get neighbours
                if child not in visit:            #if new location
                    visit.add(child)              #Add child to set of visited locations
                    front.push(Node(child,curr))  #Add child to frontier with current as parent
        return None                               #goal not found

    def bfs(self, strt: T, trgt: Callable[[T], bool], next: Callable[[T], List[T]]) -> Optional[Node[T]]:        
        front: Stack[Node[T]] = DQueue()
        visit:        Set[T]                   = {strt}
        front.push(Node(strt,None))
        while not front.isempty:                  #while we can explore
            curr: Node[T] = front.pop()           #get the first in node
            if trgt(curr.xy_loc): return curr     #goal found
            for child in next(curr.xy_loc):       #get neighbours
                if child not in visit:            #if new location
                    visit.add(child)              #Add child to set of visited locations
                    front.push(Node(child,curr))  #Add child to frontier with current as parent
        return None                               #goal not found

    def ast(self, strt:T,trgt:Callable[[T],bool],next:Callable[[T],List[T]],cost:Callable[[T],float],heur:Callable[[T],float])->Optional[Node[T]]:
        front: PQueue[Node[T]] = PQueue()
        visit: Dict[T, float]  = {strt: 0.0}
        front.push(Node(strt, None, 0.0, heur(strt)))
        while not front.isempty:                                      #while we can explore
            curr: Node[T] = front.pop()                               #get next node with least cost
            if trgt(curr.xy_loc): return curr                         #goal found
            for child in next(curr.xy_loc):                           #get neighbours
                cost_i: float = cost(curr.xy_loc)                     #cost to get here from start
                if child not in visit or visit[child]>cost_i:         #if new location or lower cost
                    visit[child] = cost_i                             #update visit with update cost
                    front.push(Node(child,curr, cost_i, heur(child))) #update front with estimated cost to get to goal

#------------------------------------------------------------------------------------------------------------------------------------------------
    def get_path(self, node: Node[T]) -> List[T]:
        path: List[T] = [node.xy_loc]
        while node.parent is not None:
            node=node.parent
            path.append(node.xy_loc)
        path.reverse()
        return path


### Maze Environment Generation & Solution

In [5]:
#------------------------------------------------------------------------------------------------------------------------------------------------
class Cell(str, Enum):
    EMPTY = "     "
    BLOCK = "  "+   "X"   + "  "
    START = "  "+   "S"   + "  "
    GOALL = "  "+   "G"   + "  "
    TRAVLD = colored("  "+"\u058d" + "  ", 'red')
    TRAVLB = colored("  "+"\u058d" + "  ", 'blue')
    TRAVLA = colored("  "+"\u058d" + "  ", 'green')

class Location(NamedTuple):
    row: int
    col: int
    
class Maze:
#------------------------------------------------------------------------------------------------------------------------------------------------   
    def __init__(self,rows:int=10,cols:int=10,blks:float=0.2,strt:Location=Location(0,0),trgt:Location=Location(0,0))->None:
        self.cloc: Location                     = strt
        self.blks: float                        = blks
        self.rows: int                          = rows
        self.cols: int                          = cols
        self.strt: Location                     = strt
        if (trgt.row, trgt.col)                == (0,0):
            self.trgt: Location                 = Location(self.rows-1, self.cols-1)
        else: self.trgt: Location               = trgt
        self.grid: List[List[Cell]]             = self.Init_Grid_Empty(self.rows, self.cols) #Create Empty Maze
        self.grid[0][0]                         = Cell.START 
        self.grid[self.rows-1][self.cols-1]     = Cell.GOALL
        self.Init_Fill_Block(self.rows,self.rows)                                            #Fill Barriers with probability of 25%

    def Init_Grid_Empty(self, num_rows, num_cols) -> List[List[Cell]]: 
        return [[Cell.EMPTY for c in range(num_cols)] for r in range(num_rows)]

    def Remv_Fill_Block(self, num_rows, num_cols) -> List[List[Cell]]: 
        for r in range(num_rows):
            for c in range(num_cols):
                if self.grid[r][c] == Cell.BLOCK:  self.grid[r][c] = Cell.EMPTY
                if self.grid[r][c] == Cell.TRAVLA: self.grid[r][c] = Cell.EMPTY
        self.grid[self.cloc.row][self.cloc.col]==  Cell.TRAVLA
    
    def Init_Fill_Block(self, num_rows, num_cols) -> List[List[Cell]]: 
        for r in range(num_rows):
            for c in range(num_cols):
                if not((r==0 and c==0) or (r==(self.rows-1) and c==(self.cols-1))):          #dont block the start/target location
                    if not(Location(r,c) == self.cloc):                                      #dont block my currentlocation
                        if random.uniform(0,1.) < self.blks: self.grid[r][c] = Cell.BLOCK

    def __repr__(self) -> str:
        ret_val: str = " "
        rep_str      = "  " + "-" +"  "
        tmp_str      = rep_str*self.cols+"\n"
        ret_val      = ret_val + tmp_str
        for row in self.grid: ret_val  = ret_val + "|" + "".join([cell.value for cell in row]) + "|\n " + tmp_str
        return ret_val

    def on_target(self, loc: Location) -> bool:
        return loc == self.trgt

    def is_block(self, loc: Location) -> bool:
        return self.grid[loc.row][loc.col] == Cell.BLOCK
        
    def get_neighbours(self, loc: Location) -> List[Location]:                   
        ret_val: List[Location] = []                                                                                           #empty list
        if loc.row+1 <self.rows and self.grid[loc.row+1][loc.col]  !=Cell.BLOCK:ret_val.append(Location(loc.row+1, loc.col))   #down  neighbour
        if loc.row-1>=        0 and self.grid[loc.row-1][loc.col]  !=Cell.BLOCK:ret_val.append(Location(loc.row-1, loc.col))   #up    neighbour
        if loc.col+1 <self.cols and self.grid[loc.row]  [loc.col+1]!=Cell.BLOCK:ret_val.append(Location(loc.row,   loc.col+1)) #right neighbour
        if loc.col-1>=        0 and self.grid[loc.row]  [loc.col-1]!=Cell.BLOCK:ret_val.append(Location(loc.row,   loc.col-1)) #left  neighbour
        return ret_val

    def show_path(self, path: List[Location], color: Optional[int] = 0) -> None:
        match color:
            case 0: 
                for loc in path: self.grid[loc.row][loc.col] = Cell.TRAVLD #set cells trarversed in color red
            case 1: 
                for loc in path: self.grid[loc.row][loc.col] = Cell.TRAVLB #set cells trarversed in color blue
            case 2: 
                for loc in path: self.grid[loc.row][loc.col] = Cell.TRAVLA #set cells trarversed in color green        
        self.grid[0][0]                     = Cell.START          #reset the start 
        self.grid[self.rows-1][self.cols-1] = Cell.GOALL          #reset the target

    def hide_path(self, path: List[Location]) -> None:
        for loc in path: self.grid[loc.row][loc.col] = Cell.EMPTY #reset all cells
        self.grid[0][0]                     = Cell.START          #reset the start 
        self.grid[self.rows-1][self.cols-1] = Cell.GOALL          #reset the target
#------------------------------------------------------------------------------------------------------------------------------------------------
    def l2_norm(self, trgt: Location) -> Callable[[Location], float]:
        def get_norm(loc: Location) -> float:
            x = loc.col - trgt.col
            y = loc.row - trgt.row
            return ((x*x)+(y*y))**0.5
        return get_norm
    def l1_norm(self, strt: Location) -> Callable[[Location], float]:
        def get_norm(loc: Location) -> float:
            x = loc.col - strt.col
            y = loc.row - strt.row
            return abs(x)+abs(y)
        return get_norm
#------------------------------------------------------------------------------------------------------------------------------------------------


In [7]:
MAZE_HEIGHT = 20
MAZE_WIDTH  = 20
BLOCK_PROB  = .2

if __name__ == "__main__":
#------------------------------------------------------------------------------------------------------------------------------------------------ 
    M = Maze(MAZE_HEIGHT,MAZE_WIDTH,BLOCK_PROB); print(M);
    S = Search_Utility()
    loop_count = 1 #just using this to flash the solution
#------------------------------------------------------------------------------------------------------------------------------------------------
    #Show DFS:
    P1: Optional[Node[Location]] = S.dfs(M.strt, M.on_target, M.get_neighbours)
    if P1 is None: printmd("DFS   Path Not Found")
    else: 
        P1 = S.get_path(P1)
        for i in range(0,loop_count):
            for p in P1:
                clear_output(wait=True)
                M.show_path([p], 0);print(M);time.sleep(0.0625);
            if i<loop_count-1:M.hide_path(P1)
#------------------------------------------------------------------------------------------------------------------------------------------------
    #Show BFS:
    P2: Optional[Node[Location]] = S.bfs(M.strt, M.on_target, M.get_neighbours)
    if P2 is None: printmd("BFS   Path Not Found")
    else: 
        P2 = S.get_path(P2)
        for i in range(0,loop_count):
            for p in P2:
                clear_output(wait=True)
                M.show_path([p], 1);print(M);time.sleep(0.0625);
            if i<loop_count-1:M.hide_path(P2)
#------------------------------------------------------------------------------------------------------------------------------------------------
    #Show AStar:
    cost: Callable[[Location], float] = M.l1_norm(M.strt)
    heur: Callable[[Location], float] = M.l2_norm(M.trgt)
    P3: Optional[Node[Location]] = S.ast(M.strt, M.on_target, M.get_neighbours, cost, heur)
    if P3 is None: printmd("AStar Path Not Found")
    else: 
        P3 = S.get_path(P3)
        for i in range(0,loop_count):
            for p in P3:
                clear_output(wait=True)
                M.show_path([p], 2);print(M);time.sleep(0.125);
            if i<loop_count-1:M.hide_path(P3)
#------------------------------------------------------------------------------------------------------------------------------------------------
    DFS_PATH = colored("\u058d" + ":", 'red')
    BFS_PATH = colored("\u058d" + ":", 'blue')
    AST_PATH = colored("\u058d" + ":", 'green')
    
    if P1 is not None: print(DFS_PATH+" Depth First Search Path")
    if P2 is not None: print(BFS_PATH+" Breadth First Search Path")
    if P3 is not None: print(AST_PATH+" A Star Search Path, using L1 Norm from Start as Cost and L2 Norm to Target as Heuristic")

   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  S    ֍    ֍    ֍    ֍    X    ֍    ֍    ֍    ֍    ֍    ֍    ֍    ֍    ֍    X              X       |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  ֍    ֍    ֍         ֍    ֍    ֍    X    X         X                   ֍    ֍    ֍    X            |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  ֍    ֍    ֍    ֍                        X              X    X                   ֍    ֍    ֍    ֍  |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  X    ֍    ֍    ֍              X         X    ֍    ֍    ֍    ֍    ֍         X              X    ֍  |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|       X    ֍    ֍    X              X    ֍    ֍              X    ֍    ֍    

### Test A-Star in a Dynamic Maze

In [8]:
MAZE_HEIGHT = 20
MAZE_WIDTH  = 20
BLOCK_PROB  = .4

if __name__ == "__main__":
#------------------------------------------------------------------------------------------------------------------------------------------------ 
    M = Maze(MAZE_HEIGHT,MAZE_WIDTH,BLOCK_PROB); print(M);
    S = Search_Utility()
    loop_count = 1 #just using this to flash the solution
#------------------------------------------------------------------------------------------------------------------------------------------------
    path: List[Location] = [M.strt]; i=0;j=0
    while not M.on_target(M.strt):
        cost: Callable[[Location], float] = M.l1_norm(M.strt)
        heur: Callable[[Location], float] = M.l2_norm(M.trgt)
        Pstr: Optional[Node[Location]]    = S.ast(M.strt, M.on_target, M.get_neighbours, cost, heur)
        if Pstr is None: 
            clear_output(wait=True)
            print(i, None);j+=1
            M.Remv_Fill_Block(M.rows,M.rows)
            M.Init_Fill_Block(M.rows,M.rows)
            print(M);time.sleep(0.0625);
            continue
        clear_output(wait=True)
        Pstr = S.get_path(Pstr)
        i+=1;print(i, Pstr[1])
        M.show_path([Pstr[1]], 2);print(M);time.sleep(0.5);
        M.strt = Pstr[1]
        path.append(Pstr[1])
        M.Remv_Fill_Block(M.rows,M.rows)
        M.Init_Fill_Block(M.rows,M.rows)
    clear_output(wait=True)
    M.show_path(path,2);print("Agent Found Path in",i,"steps, and loitered for",j,"steps");print(M);time.sleep(1)
#------------------------------------------------------------------------------------------------------------------------------------------------

    AST_PATH = colored("\u058d" + ":", 'green')
    print(AST_PATH+" A Star Search Path, using L1 Norm from Start as Cost and L2 Norm to Target as Heuristic in Dynamic Maze")

Agent Found Path in 76 steps, and loitered for 289 steps
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  S    X    ֍              X                   X         X         X              X              X  |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  ֍    ֍    ֍    ֍    ֍         X              X         X                                          |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  X                   ֍    ֍    ֍    X    X         X    X         X         X    X         X    X  |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|                 X    ֍    ֍    ֍              X         X         X                                |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|       X            

### Instead of Loitering when no path found Move to Neighbour with Lowest Cost

In [21]:
MAZE_HEIGHT = 20
MAZE_WIDTH  = 20
BLOCK_PROB  = .5

if __name__ == "__main__":
#------------------------------------------------------------------------------------------------------------------------------------------------ 
    M = Maze(MAZE_HEIGHT,MAZE_WIDTH,BLOCK_PROB); print(M);
    S = Search_Utility()
    loop_count = 1 #just using this to flash the solution
#------------------------------------------------------------------------------------------------------------------------------------------------
    path: List[Location] = [M.strt]; i=0;j=0
    Pstr: Optional[Node[Location]] = []
    while not M.on_target(M.strt):
        cost: Callable[[Location], float] = M.l1_norm(M.strt)
        heur: Callable[[Location], float] = M.l2_norm(M.trgt)
        Pstr: Optional[Node[Location]]    = S.ast(M.strt, M.on_target, M.get_neighbours, cost, heur)
        if Pstr is None: 
            nay:List[Location] = M.get_neighbours(M.strt)
            if len(nay)>0:
                nay_cost = [heur(loc) for loc in nay]
                min_cost = min(range(len(nay_cost)), key=nay_cost.__getitem__)
                if heur(M.strt)<nay_cost[min_cost]: #this will help reduce backtracking.. 
                    nex_step = M.strt; j+=1;
                else: nex_step = nay[min_cost]
                Pstr     = [M.strt, nex_step]
            else:
                clear_output(wait=True)
                print(i, None);j+=1
                M.Remv_Fill_Block(M.rows,M.rows)
                M.Init_Fill_Block(M.rows,M.rows)
                print(M);time.sleep(0.0625);
                continue
        else: Pstr = S.get_path(Pstr)
        clear_output(wait=True)
        i+=1;print(i, Pstr[1])
        M.show_path([Pstr[1]], 2);print(M);time.sleep(0.25);
        M.strt = Pstr[1]
        path.append(Pstr[1])
        M.Remv_Fill_Block(M.rows,M.rows)
        M.Init_Fill_Block(M.rows,M.rows)
    clear_output(wait=True)
    M.show_path(path,2);print("Agent Found Path in",i,"steps, and loitered for",j,"steps");print(M);time.sleep(1)
#------------------------------------------------------------------------------------------------------------------------------------------------
    AST_PATH = colored("\u058d" + ":", 'green')
    print(AST_PATH+" A Star Search Path, using L1 Norm from Start as Cost and L2 Norm to Target as Heuristic in Dynamic Maze")

Agent Found Path in 55 steps, and loitered for 12 steps
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  S    ֍    X    X         X                   X                   X    X    X    X    X            |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  X    ֍    ֍    X         X    X                        X    X    X    X         X              X  |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|            ֍              X              X              X    X         X    X         X         X  |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  X         ֍    ֍    ֍    ֍    X         X    X         X              X    X    X         X    X  |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|       X             

### Instead of Calling A-Star Continuously, Continue on path until next location is blocked.

In [61]:
MAZE_HEIGHT = 20
MAZE_WIDTH  = 20
BLOCK_PROB  = .5

if __name__ == "__main__":
#------------------------------------------------------------------------------------------------------------------------------------------------ 
    M = Maze(MAZE_HEIGHT,MAZE_WIDTH,BLOCK_PROB); print(M);
    S = Search_Utility()
    loop_count = 1 #just using this to flash the solution
#------------------------------------------------------------------------------------------------------------------------------------------------
    path: List[Location]              = [M.strt]; i=0;j=0
    cost: Callable[[Location], float] = M.l1_norm(M.strt)
    heur: Callable[[Location], float] = M.l2_norm(M.trgt)
    Apth: List[Location]              = []
    
    while not M.on_target(M.strt):
        pblk = M.is_block(Apth[1]) if len(Apth)>1 else False
        if len(Apth)<2 or pblk:
            Pstr: Optional[Node[Location]] = S.ast(M.strt, M.on_target, M.get_neighbours, cost, heur)
            if Pstr is None:
                nay:List[Location]            = M.get_neighbours(M.strt); nay.append(M.strt)
                nay_cost                      = [heur(loc) for loc in nay]
                min_cost                      = min(range(len(nay_cost)), key=nay_cost.__getitem__)
                Apth                          = [M.strt, nay[min_cost]]
                if min_cost==(len(nay_cost)-1): j=j+1;i=i-1;
            else:
                Apth = S.get_path(Pstr)
        clear_output(wait=True)
        i+=1;print(i, Apth[1])
        M.show_path([Apth[1]], 2);print(M);time.sleep(0.25);
        M.strt = Apth[1]
        path.append(Apth[1])
        M.Remv_Fill_Block(M.rows,M.rows)
        M.Init_Fill_Block(M.rows,M.rows)
    clear_output(wait=True)
    M.show_path(path,2);print("Agent Found Path in",i,"steps, and loitered for",j,"steps");print(M);time.sleep(1)
#------------------------------------------------------------------------------------------------------------------------------------------------
    AST_PATH = colored("\u058d" + ":", 'green')
    print(AST_PATH+" A Star Search Path, using L1 Norm from Start as Cost and L2 Norm to Target as Heuristic in Dynamic Maze")

Agent Found Path in 105 steps, and loitered for 10 steps
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|  S    ֍    X    X    X         X    X    X    X              X         X         X    X    X       |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|       ֍    X    X    X    X                   X    X    X    X    X    X    X    X    X            |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|       ֍    ֍    ֍         X                        X         X         X         X    X    X    X  |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|            X    ֍              X    X    X    X              X    X    X         X         X       |
   -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -  
|            X    ֍  